In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
import tensorflow as tf

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

In [4]:
train_dir = '/Users/jeffreyscruggs/Desktop/DataSets/chest_xray/train'
validation_dir = '/Users/jeffreyscruggs/Desktop/DataSets/chest_xray/test'

In [5]:
train_NORMAL_dir = os.path.join(train_dir, 'NORMAL') # directory with our training elephant pictures
train_PNEUMONIA_dir = os.path.join(train_dir, 'PNEUMONIA') # directory with our training tiger pictures
validation_NORMAL_dir = os.path.join(validation_dir, 'NORMAL') # directory with our validation elephant pictures
validation_PNEUMONIA_dir = os.path.join(validation_dir, 'PNEUMONIA') # directory with our validation tiger pictures

In [6]:
num_NORMAL_tr = len(os.listdir(train_NORMAL_dir))
num_PNEUMONIA_tr = len(os.listdir(train_PNEUMONIA_dir))
num_NORMAL_val = len(os.listdir(validation_NORMAL_dir))
num_PNEUMONIA_val = len(os.listdir(validation_PNEUMONIA_dir))
total_train = num_NORMAL_tr + num_PNEUMONIA_tr
total_val = num_NORMAL_val + num_PNEUMONIA_val
print('total training NORMAL images:', num_NORMAL_tr)
print('total training PNEUMONIA images:', num_PNEUMONIA_tr)

print('total validation NORMAL images:', num_NORMAL_val)
print('total validation PNEUMONIA images:', num_PNEUMONIA_val)
print("--")
print("Total training images:", total_train)
print("Total validation images:", total_val)

total training NORMAL images: 1341
total training PNEUMONIA images: 3875
total validation NORMAL images: 234
total validation PNEUMONIA images: 390
--
Total training images: 5216
Total validation images: 624


In [7]:
batch_size = 128
epochs = 5
IMG_HEIGHT = 150
IMG_WIDTH = 150

In [8]:
train_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data

In [9]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
 directory=train_dir,
 shuffle=True,
 target_size=(IMG_HEIGHT, IMG_WIDTH),
 class_mode='binary')
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
 directory=validation_dir,
 target_size=(IMG_HEIGHT, IMG_WIDTH),
 class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [10]:
model = Sequential([
 Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
 MaxPooling2D(),
 Conv2D(32, 3, padding='same', activation='relu'),
 MaxPooling2D(),
 Conv2D(64, 3, padding='same', activation='relu'),
 MaxPooling2D(),
 Flatten(),
 Dense(512, activation='relu'),
 Dense(1, activation='sigmoid')
])

In [11]:
model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=['accuracy'])

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 37, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 18, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 20736)             0

In [13]:
history = model.fit_generator(
 train_data_gen,
 steps_per_epoch= total_train // batch_size,
 epochs=epochs,
 validation_data= val_data_gen,
 validation_steps= total_val // batch_size
)

Epoch 1/5
40/40 [==============================] - 108s 3s/step - loss: 0.3683 - accuracy: 0.8365 - val_loss: 0.4783 - val_accuracy: 0.8320
Epoch 2/5
40/40 [==============================] - 108s 3s/step - loss: 0.1175 - accuracy: 0.9560 - val_loss: 0.9520 - val_accuracy: 0.7246
Epoch 3/5
40/40 [==============================] - 109s 3s/step - loss: 0.0850 - accuracy: 0.9674 - val_loss: 0.6725 - val_accuracy: 0.7695
Epoch 4/5
40/40 [==============================] - 109s 3s/step - loss: 0.0774 - accuracy: 0.9727 - val_loss: 1.0122 - val_accuracy: 0.7422
Epoch 5/5
40/40 [==============================] - 113s 3s/step - loss: 0.0691 - accuracy: 0.9744 - val_loss: 1.0874 - val_accuracy: 0.7363


In [14]:
tf.__version__

'2.0.0'

In [15]:
model.save('PNEUMONIA_Beta')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: PNEUMONIA_Beta/assets


In [16]:
print(train_data_gen.class_indices)

{'NORMAL': 0, 'PNEUMONIA': 1}


In [17]:
import numpy as np
from tensorflow.keras.preprocessing import image
from PIL import Image, ImageFont, ImageDraw


In [20]:
test_image = image.load_img('/Users/jeffreyscruggs/Desktop/DataSets/chest_xray/val/NORMAL/NORMAL2-IM-1437-0001.jpeg', target_size = (150,150))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)
print(result)

[[0.]]
